In [11]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill

# Define department-college mapping (for demo purposes, you can expand this as needed)
department_college_map = {
    "MATH": ("Mathematics Department", "Science College"),
    "BIO": ("Biology Department", "Science College"),
    "PHY": ("Physics Department", "Science College"),
    "CHEM": ("Chemistry Department", "Science College"),
    "COMP": ("Computer Science Department", "Engineering College"),
    # Add more mappings as required
}

def extract_level(course_code):
    """Extract the level from the course code."""
    if pd.isna(course_code):
        return None
    if "/" in course_code:  # For combined courses
        course_code = course_code.split("/")[0]
    digits = ''.join(filter(str.isdigit, course_code))
    if digits and len(digits) >= 3:
        return int(digits[0])  # The first digit indicates the level
    return None

def process_courses(sheet_path):
    df = pd.read_excel(sheet_path, sheet_name="Second Semester")
    course_data = {}

    for _, row in df.iterrows():
        course_code = row["COURSE"]
        programs = row["PROGRAMMES OFFERING"]
        exam_type = row["TYPE"]
        level = extract_level(course_code)

        # Handle missing "PROGRAMMES OFFERING" values
        if pd.isna(programs):
            programs = ""

        # Check if there are multiple programs (comma-separated) in the field
        if ',' in programs:
            program_list = [program.strip() for program in programs.split(",")]
        else:
            program_list = [programs.strip()]
        print(program_list)
        departments, colleges = [], []
        for program in program_list:
            # Determine department and college based on program code
            department, college = "Unknown Department", "Unknown College"
            for code, (dept, coll) in department_college_map.items():
                if code in program:
                    department, college = dept, coll
                    break
            departments.append(department)
            colleges.append(college)

        # Aggregate the data by course code
        if course_code not in course_data:
            course_data[course_code] = {
                "Programs": [],
                "Exam Type": exam_type,
                "Level": level,
                "Departments": [],
                "Colleges": [],
            }

        course_data[course_code]["Programs"].extend(program_list)
        course_data[course_code]["Departments"].extend(departments)
        course_data[course_code]["Colleges"].extend(colleges)

    # Consolidate the data into a DataFrame
    processed_courses = []
    for course_code, data in course_data.items():
        processed_courses.append({
            "Course Code": course_code,
            "Program": ", ".join(sorted(set(data["Programs"]))),
            "Exam Type": data["Exam Type"],
            "Level": data["Level"],
            "Department": ", ".join(
                ["Unknown Department" if dept == "Unknown Department" else dept for dept in sorted(set(data["Departments"]))]
            ),
            "College": ", ".join(
                ["Unknown College" if coll == "Unknown College" else coll for coll in sorted(set(data["Colleges"]))]
            ),
        })

    return pd.DataFrame(processed_courses)



def process_venues(sheet_path):
    """Process the 'Venues' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="Venues")
    return df.rename(columns={"HALL": "Venue", "CODE": "Code", "EXAM CAPACITY": "Capacity"})

def process_staff(sheet_path):
    """Process the 'List of Staff' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="List of Staff")
    return df.rename(columns={"NAME": "Name", "COLLEGE": "College", "DEPARTMENT": "Department", "PHONE NUMBER": "Phone"})

def adjust_column_width_and_style(writer, sheet_name):
    """Adjust column width and apply header style in Excel."""
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Adjust column widths
    for column_cells in worksheet.columns:
        max_length = max((len(str(cell.value)) for cell in column_cells if cell.value), default=0)
        adjusted_width = max_length + 4  # Add some spacing
        worksheet.column_dimensions[column_cells[0].column_letter].width = adjusted_width

    # Apply header style (yellow background, bold text, centered alignment)
    header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    header_font = Font(bold=True)
    header_alignment = Alignment(horizontal="center", vertical="center")

    for cell in worksheet[1]:  # First row (header)
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment

def main(input_file, output_file):
    # Process data from Excel sheets
    courses_df = process_courses(input_file)
    venues_df = process_venues(input_file)
    staff_df = process_staff(input_file)

    # Write the processed data to a new Excel file
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        courses_df.to_excel(writer, sheet_name="Courses", index=False)
        adjust_column_width_and_style(writer, "Courses")

        venues_df.to_excel(writer, sheet_name="Venues", index=False)
        adjust_column_width_and_style(writer, "Venues")

        staff_df.to_excel(writer, sheet_name="Staff", index=False)
        adjust_column_width_and_style(writer, "Staff")

    print(f"Processed data saved to {output_file}")

if __name__ == "__main__":
    # Input and output files
    input_excel = "Data for Exam timetable.xlsx"  # Replace with your actual file name
    output_excel = "data10.xlsx"

    if os.path.exists(input_excel):
        main(input_excel, output_excel)
    else:
        print(f"Input file '{input_excel}' not found. Please provide the correct file.")


In [7]:
import pandas as pd
import os
from datetime import datetime


def generate_timetable(data_file, output_file):
    # Read the processed courses data
    courses_df = pd.read_excel(data_file, sheet_name="Courses")
    timetable_cbt = []
    timetable_written = []

    # Example invigilator ID assignment
    invigilator_id = 1

    # Iterate over the courses
    for _, row in courses_df.iterrows():
        course_code = row["Course Code"]
        programs = row["Program"]
        exam_type = row["Exam Type"]
        level = row["Level"]
        department = row["Department"]
        college = row["College"]

        # Number of students is estimated based on a fixed count per program
        # (This can be replaced with actual student count data if available)
        student_count = len(programs.split(", ")) * 40  # Assume 40 students per program
        invigilators_needed = (student_count // 40) + (1 if student_count % 40 > 0 else 0)

        # Generate time slots (example implementation for demo purposes)
        time_slots = [
            ("08:00 AM - 10:00 AM", 0),
            ("10:00 AM - 12:00 PM", 1),
            ("02:00 PM - 04:00 PM", 2),
        ]
        exam_date = datetime(2024, 7, 29)  # Example start date
        for idx, (time, session_idx) in enumerate(time_slots):
            date = (exam_date + pd.Timedelta(days=session_idx)).strftime("%d/%m/%y")
            day = (exam_date + pd.Timedelta(days=session_idx)).strftime("%A")

            # Assign invigilators
            invigilators = []

            if exam_type.lower() == "cbt":
                invigilators = [
                    f"C1-{invigilator_id}",
                    f"L1-{invigilator_id + 1}",
                    f"L2-{invigilator_id + 2}",
                    f"L3-{invigilator_id + 3}",
                    f"L4-{invigilator_id + 4}",
                    f"E-{invigilator_id + 5}",
                ]
                invigilator_id += 6
                timetable_cbt.append({
                    "Date": date,
                    "Day": day,
                    "Time": time,
                    "Course": course_code,
                    "Programmes": programs,
                    "Invigilator Numbers": ", ".join(invigilators),
                })
            else:  # Written exam
                for _ in range(invigilators_needed):
                    invigilators.append(f"WR-{invigilator_id}")
                    invigilator_id += 1
                timetable_written.append({
                    "Date": date,
                    "Day": day,
                    "Time": time,
                    "Course": course_code,
                    "Programmes": programs,
                    "Departments": department,
                    "Colleges": college,
                    "Invigilator Numbers": ", ".join(invigilators),
                })

    # Save both timetables to the output file
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        pd.DataFrame(timetable_cbt).to_excel(writer, sheet_name="CBT Timetable", index=False)
        pd.DataFrame(timetable_written).to_excel(writer, sheet_name="Written Timetable", index=False)

    print(f"Timetables saved to {output_file}")


if __name__ == "__main__":
    input_file = "data10.xlsx"  # Input file from Script 1
    output_file = "Exam_Timetable.xlsx"

    if os.path.exists(input_file):
        generate_timetable(input_file, output_file)
    else:
        print(f"Input file '{input_file}' not found. Please provide the correct file.")


AttributeError: 'float' object has no attribute 'lower'